# $\Psi$ Method

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import csv

def GAUSSIAN(SOA,PSS,TBW,h=1):
    
    SD = (TBW-PSS)/np.log(4)
    
    g = h * np.exp(-((SOA-PSS)/SD)**2/2)
    
    return g

def SIMULATE_OBSERVER(SOA_next):
    PSS = 5
    TBW = 140
    p_next_true = GAUSSIAN(SOA_next,PSS,TBW)
    response = np.random.choice([0, 1], p=[1-p_next_true, p_next_true])
    
    return response

nSOA = 1000
nVal = 20
nPar = 2 # PSS, TBW
    
# SOA values
SOA_space = np.linspace(-200,200,nSOA) # ms
    
# All possible values of parameters (PSS, TBW)...
theta_space = np.zeros((nVal,nPar))
theta_space[:,0] = np.linspace(-10,10,nVal)     # PSS
theta_space[:,1] = np.linspace(120,240,nVal)    # TBW

# ...and combinations (param vectors)
nVec = nVal**nPar
theta_vector = np.zeros((nVec,nPar))
for p in range(nPar):
    theta_vector[:,p] = np.tile(np.repeat(theta_space[:,p],nVal**(nPar-p-1)),nVal**p)

# Probability of response (success) to stimulus, given parameters (theta)... p(failure) = 1-p(success)
gauss = np.zeros((nSOA,nVec))
for i,[PSS,TBW] in enumerate(theta_vector):
    gauss[:,i] = GAUSSIAN(SOA_space,PSS,TBW)
    
prior = np.ones(nVec)/nVec

# List of posterior update and expectd entropy
ENTROPY = []

nTrial = 30

for t in range(nTrial):
    
    # 1)
    p_success = np.matmul(gauss,prior)
    p_failure = 1-p_success
    
    # 2)
    posterior_success = np.zeros((nSOA,nVec))
    posterior_failure = np.zeros((nSOA,nVec))
    for i in range(nSOA):
        for j in range(nVec):
            posterior_success[i,j] = gauss[i,j]*prior[j]/p_success[i]
            posterior_failure[i,j] = (1-gauss[i,j])*prior[j]/p_failure[i]

    # 3)
    post_entropy_success = np.zeros(nSOA)
    post_entropy_failure = np.zeros(nSOA) 
    for i in range(nSOA):
        post_entropy_success[i] = -sum(posterior_success[i,:]*np.log(posterior_success[i,:]))
        post_entropy_failure[i] = -sum(posterior_failure[i,:]*np.log(posterior_failure[i,:]))
        
    # 4)
    expected_post_entropy = p_success*post_entropy_success + p_failure*post_entropy_failure
    ENTROPY.append(expected_post_entropy)
    
    # 5)
    ind_next = np.argmin(expected_post_entropy)
    SOA_next = SOA_space[ind_next]
    
    # 6)
    response = SIMULATE_OBSERVER(SOA_next)
    
    # 7)
    if response:
        prior = gauss[ind_next,:]*prior/p_success[ind_next]
    else:
        prior = (1-gauss[ind_next,:])*prior/p_failure[ind_next]
        
p_theta = np.zeros((nVal,nPar))

for i in range(nVal):
        for p in range(nPar):
            ind = np.arange(nVec/nVal**(nPar-p-1))
            tmp = np.repeat(ind%nVal,nVal**(nPar-p-1))
            x = np.argwhere(tmp==i)
            p_theta[i,p] = sum(prior[x])

# plt.subplot(121)
# plt.plot(theta_space[:,0],p_theta[:,0])
# plt.subplot(122)
# plt.plot(theta_space[:,1],p_theta[:,1])
# plt.show()

# PSS_est = theta_space[np.argmax(p_theta[:,0]),0]

PSS_est, TBW_est = sum(theta_space[:,0]*p_theta[:,0]) , sum(theta_space[:,1]*p_theta[:,1])
print(f'PSS estimated = {PSS_est:.2f}\nTBW estimated = {TBW_est:.2f}')
print(f'PSS true = 5\nTBW true = 140')

PSS estimated = -0.49
TBW estimated = 145.90
PSS true = 5
TBW true = 140


In [3]:
# results = np.concatenate((theta_space, p_theta),axis=1)

with open('ParamEst.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow([PSS_est, TBW_est])
    
with open('ParamValues.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(theta_space)

with open('ParamProb.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(p_theta)